<a href="https://colab.research.google.com/github/mohamedrafik5/AI-ML-COURSE-PROGRESS/blob/main/3_DC_Grid_search_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#importing the Libraies
import numpy as np
import pandas as pd

In [5]:
# Reading the Dataset
dataset = pd.read_csv('/content/Social_Network_Ads.csv')

In [6]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [7]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [8]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [9]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [10]:
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [12]:
from sklearn.svm import SVC

In [13]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [14]:
from sklearn.model_selection import GridSearchCV, cross_val_predict

# Define parameter grid (smaller for speed)
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto'],
    'class_weight': [None, 'balanced']
}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X,y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10], 'class_weight': [None, 'balanced'],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf']},
             scoring='f1_weighted', verbose=3)

In [15]:
# Cross-validated predictions (instead of X_test)
y_pred = cross_val_predict(grid.best_estimator_, X, y, cv=5)

In [16]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y, y_pred)
clf_report = classification_report(y, y_pred)
f1_macro = f1_score(y, y_pred, average='weighted')

In [17]:
print("Best Parameters:", grid.best_params_)
print("The f1_macro:", f1_macro)
print("The confusion Matrix:\n", cm)
print("The report:\n", clf_report)

Best Parameters: {'C': 1, 'class_weight': 'balanced', 'gamma': 'scale', 'kernel': 'rbf'}
The f1_macro: 0.9058866666666667
The confusion Matrix:
 [[231  26]
 [ 12 131]]
The report:
               precision    recall  f1-score   support

           0       0.95      0.90      0.92       257
           1       0.83      0.92      0.87       143

    accuracy                           0.91       400
   macro avg       0.89      0.91      0.90       400
weighted avg       0.91      0.91      0.91       400



In [18]:
# Save Best Model
import pickle
filename = "SVC_CV_best_model.sav"
pickle.dump(grid.best_estimator_, open(filename, 'wb'))

In [19]:
# CV Results table
results_df = pd.DataFrame(grid.cv_results_)
results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005908,0.000683,0.006203,0.000877,0.1,None,scale,linear,"{'C': 0.1, 'class_weight': None, 'gamma': 'sca...",0.559115,0.949546,0.909305,0.803125,0.708631,0.785944,0.141214,23
1,0.006122,0.000291,0.006341,0.000245,0.1,None,scale,rbf,"{'C': 0.1, 'class_weight': None, 'gamma': 'sca...",0.842857,0.937229,0.911364,0.803125,0.887896,0.876494,0.048061,11
2,0.005682,0.001821,0.009044,0.004453,0.1,None,auto,linear,"{'C': 0.1, 'class_weight': None, 'gamma': 'aut...",0.559115,0.949546,0.909305,0.803125,0.708631,0.785944,0.141214,23
3,0.007573,0.000713,0.006710,0.000659,0.1,None,auto,rbf,"{'C': 0.1, 'class_weight': None, 'gamma': 'aut...",0.842857,0.937229,0.911364,0.803125,0.887896,0.876494,0.048061,11
4,0.006779,0.000848,0.005168,0.000207,0.1,balanced,scale,linear,"{'C': 0.1, 'class_weight': 'balanced', 'gamma'...",0.690376,0.926000,0.925000,0.764618,0.703383,0.801875,0.104007,13


In [20]:
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler

# Assuming 'sc' and 'filename' are defined in previous cells and available in the environment

# Get input from the user
age = int(input("Enter Age: "))
estimated_salary = int(input("Enter Estimated Salary: "))
gender_male = int(input("Enter Gender (0 for Female, 1 for Male): "))

# Create a DataFrame with the user input (matching the training data columns)
user_data = pd.DataFrame([[age, estimated_salary, gender_male]], columns=['Age', 'EstimatedSalary', 'Gender_Male'])

# Scale the user input using the same scaler used for training
user_data_scaled = sc.transform(user_data)

# Load the saved model
loaded_model = pickle.load(open(filename, 'rb'))

# Make a prediction
prediction = loaded_model.predict(user_data_scaled)

# Print the prediction
if prediction[0] == 1:
    print("Prediction: Purchased")
else:
    print("Prediction: Not Purchased")

Enter Age: 21
Enter Estimated Salary: 200000
Enter Gender (0 for Female, 1 for Male): 1
Prediction: Purchased
